# Pregunta 1, parte 1

## Importacion

Se importa el módulo `pandas`, y se importa un módulo personal llamado `paths`, que contiene los paths que utilizaremos en este análisis.

In [ ]:
import pandas as pd

from paths import *

Esta es la documentación de `paths` (decomentar para verlas).

In [ ]:
# Documentación del módulo 'paths'.
# import paths
# help(paths)

## Carga de datos

### Creación de la tabla mezclada

Cargar todos los .csv en dos listas distinguidas por la aparición de `furnished` en el nombre.

In [ ]:
# Lista de DataFrames (Sin 'furnished' en el nombre)
L_DF_a = [pd.read_csv(dict_csv_mc_a[wNN]) for wNN in L_WNN]

# Lista de DataFrames (Con 'furnished' en el nombre)
L_DF_f = [pd.read_csv(dict_csv_mc_f[wNN]) for wNN in L_WNN]

Se unen los `DataFrames` de las listas en un único `DataFrame`, luego se crea una nueva columna llamada `'furnished'` y finalmente se unen los dos `DataFrames` en uno, con la nueva columna incluida.

In [ ]:
# se unen todos los data_frame en cada lista con el comando concat
DF_a = pd.concat(L_DF_a)
DF_f = pd.concat(L_DF_f)

# Se eliminan las filas duplicadas de cada data frame por separado
DF_a.drop_duplicates(subset=DF_a.columns, ignore_index=True, inplace=True, keep='last')
DF_f.drop_duplicates(subset=DF_f.columns, ignore_index=True, inplace=True, keep='last')

# Se crea una columna con tantos ceros como la cantidad de filas de DF_a
# y con tantos unos como la cantidad de filas de DF_f 
furnished_col = pd.DataFrame(data={'furnished':[0]*(DF_a.shape[0]) + [1]*(DF_f.shape[0])})

# Concateno los data frames DF_a y DF_f y le agrego al final la columna furnished_col
df = pd.concat([DF_a, DF_f], ignore_index=True) 
df = pd.concat([df, furnished_col], axis=1)

In [ ]:
# vemos cuantas filas tiene cada data frame (para reportar si hay filas en furnished 
# que no estan en all)
# print(DF_a.shape[0])
# print(DF_f.shape[0])
# nota: DF_a tiene 16295 filas y DF_f tiene 2099 filas  

Para terminar, se quitan las filas duplicadas en ambos grupos, furnished y all

In [ ]:
# Quitamos las filas que sean iguales en todas las columnas con excepcion de
# la columna furnished y reindexamos
df.drop_duplicates(subset=df.columns[:-1], ignore_index=True, inplace=True, keep='last')

# comentario 1: si hay filas duplicadas con distintos valores de furnished, se eliminan las primera
# comentario 2: el comentario 1 asegura que si hay dos filas iguales con distintos valores de furnished,
# se elimine la que tiene el valor 0 para furnished, lo que tiene sentido dado que furnished puede estar
# contenido en all

In [ ]:
# reportar si Reporte si existen observaciones de archivos con texto ’furnished’ que no estén
# contenidos en archivos con texto ’all’
# print(df[df['furnished'] == 0].shape[0])
# print(df[df['furnished'] == 1].shape[0])
# como ambas selecciones de filas en el dataframe tienen las mismas filas que los originales
# luego de eliminar las filas iguales, se conluye que no hay filas en furnished que no esten
# en all

## Pregunta 1, parte 2

### a) Limpieza de datos

Se crea un diccionario llamado `valores_nulos` que contendrá el nombre de las columnas de `df` como llave, y el valor de la llave será lo que reemplazará el valor `NaN`.

In [ ]:
# Creamos valores a reemplazar
valores_nulos = {
#     'property_type|rent_type|location':'Sin nombre',
    'price':'$0',
    'n_rooms':'NA',
    'n_bath':'NA',
    'surface':'0.0 m2',
#     'details':'',
#     'url':'',
#     'metrocuadrado_index':0.0,
#     'furnished':0
}

In [ ]:
# Reemplazamos los valores
df.fillna(value=valores_nulos, inplace=True)

#### Reemplazo de los tipos de algunas columnas

Además, en esta etapa se cambiará el tipo de algunas columnas.

##### Funciones auxiliares

In [ ]:
# Función que mapeará la columna `price`
def precio_to_int(s:str)->int:
    """
    Toma un precio en forma de `str`, en el formato `'$XXX.XXX.XXX'` 
    con X algún número, y retorna el número entero de ese precio.
    """
    return int(s.replace('$', '').replace('.', ''))

# Función que mapeará la columna `surface`
def n_surface(s:str)->float:
    """
    Toma un `str` en formato "XX.Xm2" y retorna el número real con los metros cuadrados.
    """
    return float(s.replace('m2', ''))

##### Reemplazo de columnas

In [ ]:
# Columna de precios
precios_col = df['price'].map(precio_to_int)
# Columna de superficie
superficie_col = df['surface'].map(n_surface)

df['price'] = precios_col
df['surface'] = superficie_col
df.rename(columns={
    'price':'price [$]',
    'surface':'surface [m2]'
}, inplace=True)

### b) Obtención de 3 columnas a partir de `'property_type|rent_type|location'`

`lower_case_col` es una columna con todas las palabras en minusculas.

In [ ]:
lower_case_col = df['property_type|rent_type|location'].str.lower()

#### Columna `tipo de inmueble`

In [ ]:
# `casa_bool_col` y `apart_bool_col` son columnas que tienen un -1 si la palabra no está 
# y un número entero positivo si la palabra está.
casa_bool_col = lower_case_col.str.find('casa')
apart_bool_col = lower_case_col.str.find('apartamento')

# Se crea una lista con la infomacion del tipo de inmueble recorriendo las dos columnas anteriores
tipo_inm_list = []
for index in range(len(lower_case_col)):
    if casa_bool_col[index] >= 0:
        tipo_inm_list.append('Casa')
    elif apart_bool_col[index] >= 0:
        tipo_inm_list.append('Apartamento')
    else:
        tipo_inm_list.append('No hay info')

# columna de tipo de inmueble
tipo_inm_col = pd.DataFrame(data = {'Tipo_de_inmueble': tipo_inm_list})

# Nota: no hay columnas que no tengan una de las dos informaciones

#### Columna `Tipo de oferta`



In [ ]:
# `vent_bool_col` y `arr_bool_col` son columnas que tienen un -1 si la palabra no está 
# y un número entero positivo si la palabra está.
vent_bool_col = lower_case_col.str.find('venta')
arr_bool_col = lower_case_col.str.find('arriendo')

# Se crea una lista con la infomacion del tipo de oferta recorriendo las dos columnas anteriores
tipo_ofer_list = []
for index in range(len(lower_case_col)):
    if (vent_bool_col[index] >= 0) and (arr_bool_col[index] >= 0):
        tipo_ofer_list.append('Arriendo y venta')
    elif arr_bool_col[index] >= 0:
        tipo_ofer_list.append('Arriendo')
    else:
        tipo_ofer_list.append('No hay info')

# columna de tipo de oferta
tipo_ofer_col = pd.DataFrame(data = {'Tipo_de_oferta': tipo_ofer_list})

# Nota: no hay columnas que no tengan una de las dos informaciones

#### Columna `location`



In [ ]:
# `sep_comas` es una serie que contiene en cada elemento una lista del string correspondiente 
# que separa por comas los grupos de strings
sep_comas = df['property_type|rent_type|location'].str.split(pat = ",")

# Se crea una lista con la infomacion location recorriendo los segundos elementos de la columna anterior
# se asume que la direccion esta al final de cada string
location_list = []
for index in range(len(sep_comas)):
    lista_strings = sep_comas[index]
    location_list.append(lista_strings[1]) 
    
# columna location
location_col = pd.DataFrame(data = {'location': location_list})

#### Agregar las 3 columnas anteriores a `df`

In [ ]:
df = pd.concat([df,tipo_inm_col], axis=1)
df = pd.concat([df,tipo_ofer_col], axis=1)
df = pd.concat([df,location_col], axis=1)

In [ ]:
# prueba de que location_col no tiene el string de arriendo (solo tiene direcciones)
# prueba_0 = location_col['location'].str.lower()
# prueba_1 = prueba_0.str.find('arriendo')
# for index in range(len(prueba_1)):
#     if prueba_1[index] >= 0:
#         print('error')

## Pregunta 1, parte 3

### Esquema para agregar las columnas `price per m2` y `garajes`

* Se desarrollarán funciones que procesen las columnas `url`.
* Se crearán nuevas columnas a partir de las columnas mapeadas a partir de las funciones creadas.
* Se agregarán estas columnas a `df`.

#### Funciones

Se detallan las funciones que mapearán las columnas a continuación:

In [ ]:
# Función que mapeará la columna `url`
def n_garaje(s:str)->str: 
    """
    Función que toma una url en forma de `str` y retorna un `str`, que será
    el número de garajes para la vivienda de esa url.
    """
    i_garaje = s.find('-garajes')
    
    # No se encuentra el garaje
    if i_garaje is -1: 
        return '0'
    
    # rescata el str con el número. Puede ser un número de la forma '4+'
    s_hasta_garaje = s[:i_garaje] # Se recortará el str hasta la aparición de '-garajes'
    i_garaje_ = s_hasta_garaje.rfind('-') + 1 # Se encontrará el índice siguiente de un guión '-'
    
    return s_hasta_garaje[i_garaje_:] # Se entrega el resto después de ese índice

#### Procesamiento de columnas

Se crearán las columnas y se procesarán para ser entregadas.

In [ ]:
#columna de los garajes
garajes_col = df['url'].map(n_garaje)

# Columna producto de la división entre las columnas de precio y superficie
p_por_s_col = df['price [$]'] / df['surface [m2]']

#### Rotulación de columnas y añadirlas a `df`

In [ ]:
df_ppm_and_g = pd.DataFrame(
    data={
        'price per m2 [$/m2]':p_por_s_col,
        'garajes':garajes_col
    }
)

df = pd.concat([df, df_ppm_and_g], axis=1)

## Pregunta 1, parte 4

### Esquema para clasificar:

In [ ]:
# crear las 8 clasificaciones con el comando query, ademas, se crean columnmas con enteros 
# que representan dicha clasificacion

# Tipo de producto 1
prod_1 = df.query('Tipo_de_inmueble == "Casa" and surface >= 80 and surface < 120', inplace = False)
list_prod_1 = [1]*(prod_1.shape[0])
# Tipo de producto 2
prod_2 = df.query('Tipo_de_inmueble == "Casa" and surface >= 120 and surface < 180', inplace = False)
list_prod_2 = [2]*(prod_2.shape[0])
# Tipo de producto 3
prod_3 = df.query('Tipo_de_inmueble == "Casa" and surface >= 180 and surface < 240', inplace = False)
list_prod_3 = [3]*(prod_3.shape[0])
# Tipo de producto 4
prod_4 = df.query('Tipo_de_inmueble == "Casa" and surface >= 240 and surface < 360', inplace = False)
list_prod_4 = [4]*(prod_4.shape[0])
# Tipo de producto 5
prod_5 = df.query('Tipo_de_inmueble == "Casa" and surface >= 360 and surface <= 460', inplace = False)
list_prod_5 = [5]*(prod_5.shape[0])
# Tipo de producto 6
prod_6 = df.query('Tipo_de_inmueble == "Apartamento" and surface >= 40 and surface < 60', inplace = False)
list_prod_6 = [6]*(prod_6.shape[0])
# Tipo de producto 7
prod_7 = df.query('Tipo_de_inmueble == "Apartamento" and surface >= 60 and surface < 80', inplace = False)
list_prod_7 = [7]*(prod_7.shape[0])
# Tipo de producto 8
prod_8 = df.query('Tipo_de_inmueble == "Apartamento" and surface >= 80 and surface <= 120', inplace = False)
list_prod_8 = [8]*(prod_8.shape[0])

In [ ]:
df